In [1]:
import re
from itertools import zip_longest

import numpy as np
from aocd.models import Puzzle
import pickle
import pyperclip

from collections import deque

In [2]:
year, day = 2024, 10

In [3]:
puzzle = Puzzle(year=year, day=day)

In [4]:
puzzle.examples[0].answer_a

'36'

In [5]:
example = puzzle.examples[0].input_data
print(example)

0123
1234
8765
9876


In [6]:
def solution_a(data: str) -> tuple[str, str, int]:
    # a good hiking trail is as long as possible and has an even, gradual, uphill slope.
    # For all practical purposes, this means that a hiking trail is any path that starts at height 0, 
    # ends at height 9, and always increases by a height of exactly 1 at each step. 
    # Hiking trails never include diagonal steps - only up, down, left, or right (from the perspective of the map).
    # A trailhead is any position that starts one or more hiking trails - here, 
    # these positions will always have height 0. 
    # A trailhead's score is the number of 9-height positions reachable from that trailhead via a hiking trail.
    hiking_map = np.array([list(map(int, list(line))) for line in data.splitlines()])
    n, c = hiking_map.shape
    print(hiking_map)
    score = 0
    
    trailheads = np.argwhere(hiking_map == 0)

    def reachable_peaks(start_pos):
        peaks = set()
        points_to_visit = deque([start_pos])
    
        while points_to_visit:
            current_pos = points_to_visit.pop()
            height = hiking_map[tuple(current_pos)]
    
            for x,y in [(-1,0), (1,0), (0,-1), (0,1)]:
                new_pos = current_pos + np.array([x,y])
    
                if 0 <= new_pos[0] < n and 0 <= new_pos[1] < c:
                    if hiking_map[tuple(new_pos)] == 9 and height==8:
                        peaks.add(tuple(new_pos))
                    elif hiking_map[tuple(new_pos)] == height + 1:
                        points_to_visit.append(new_pos)

        return peaks

    peaks_per_trailhead = [(trailhead, reachable_peaks(trailhead)) for trailhead in trailheads]

    # print out reachable peaks
    hiking_map = hiking_map.astype(str)
    for trailhead, peaks in peaks_per_trailhead:
        for peak in peaks:
            hiking_map[tuple(peak)] = "X"
    print(hiking_map)

    return sum(len(x[1]) for x in peaks_per_trailhead)

In [7]:
solution_a(puzzle.examples[0].input_data)

[[0 1 2 3]
 [1 2 3 4]
 [8 7 6 5]
 [9 8 7 6]]
[['0' '1' '2' '3']
 ['1' '2' '3' '4']
 ['8' '7' '6' '5']
 ['X' '8' '7' '6']]


1

In [8]:
assert solution_a(puzzle.examples[0].input_data) == 1

[[0 1 2 3]
 [1 2 3 4]
 [8 7 6 5]
 [9 8 7 6]]
[['0' '1' '2' '3']
 ['1' '2' '3' '4']
 ['8' '7' '6' '5']
 ['X' '8' '7' '6']]


In [9]:
input_ = """...0...
...1...
...2...
6543456
7.....7
8.....8
9.....9""".replace(".","3")
assert solution_a(input_) == 2

[[3 3 3 0 3 3 3]
 [3 3 3 1 3 3 3]
 [3 3 3 2 3 3 3]
 [6 5 4 3 4 5 6]
 [7 3 3 3 3 3 7]
 [8 3 3 3 3 3 8]
 [9 3 3 3 3 3 9]]
[['3' '3' '3' '0' '3' '3' '3']
 ['3' '3' '3' '1' '3' '3' '3']
 ['3' '3' '3' '2' '3' '3' '3']
 ['6' '5' '4' '3' '4' '5' '6']
 ['7' '3' '3' '3' '3' '3' '7']
 ['8' '3' '3' '3' '3' '3' '8']
 ['X' '3' '3' '3' '3' '3' 'X']]


In [10]:
input_ = """..90..9
...1.98
...2..7
6543456
765.987
876....
987....""".replace(".", "3")
assert solution_a(input_) == 4

[[3 3 9 0 3 3 9]
 [3 3 3 1 3 9 8]
 [3 3 3 2 3 3 7]
 [6 5 4 3 4 5 6]
 [7 6 5 3 9 8 7]
 [8 7 6 3 3 3 3]
 [9 8 7 3 3 3 3]]
[['3' '3' '9' '0' '3' '3' 'X']
 ['3' '3' '3' '1' '3' 'X' '8']
 ['3' '3' '3' '2' '3' '3' '7']
 ['6' '5' '4' '3' '4' '5' '6']
 ['7' '6' '5' '3' 'X' '8' '7']
 ['8' '7' '6' '3' '3' '3' '3']
 ['X' '8' '7' '3' '3' '3' '3']]


In [11]:
input_ = """89010123
78121874
87430965
96549874
45678903
32019012
01329801
10456732"""
solution_a(input_) == 36

[[8 9 0 1 0 1 2 3]
 [7 8 1 2 1 8 7 4]
 [8 7 4 3 0 9 6 5]
 [9 6 5 4 9 8 7 4]
 [4 5 6 7 8 9 0 3]
 [3 2 0 1 9 0 1 2]
 [0 1 3 2 9 8 0 1]
 [1 0 4 5 6 7 3 2]]
[['8' 'X' '0' '1' '0' '1' '2' '3']
 ['7' '8' '1' '2' '1' '8' '7' '4']
 ['8' '7' '4' '3' '0' 'X' '6' '5']
 ['X' '6' '5' '4' 'X' '8' '7' '4']
 ['4' '5' '6' '7' '8' 'X' '0' '3']
 ['3' '2' '0' '1' 'X' '0' '1' '2']
 ['0' '1' '3' '2' 'X' '8' '0' '1']
 ['1' '0' '4' '5' '6' '7' '3' '2']]


True

In [12]:
answer_a = solution_a(puzzle.input_data)

[[1 4 5 ... 0 3 6]
 [0 3 4 ... 1 4 5]
 [1 2 5 ... 2 3 4]
 ...
 [2 9 6 ... 6 5 0]
 [1 2 5 ... 7 8 9]
 [0 3 4 ... 6 7 8]]
[['1' '4' '5' ... '0' '3' '6']
 ['0' '3' '4' ... '1' '4' '5']
 ['1' '2' '5' ... '2' '3' '4']
 ...
 ['2' 'X' '6' ... '6' '5' '0']
 ['1' '2' '5' ... '7' '8' 'X']
 ['0' '3' '4' ... '6' '7' '8']]


In [15]:
answer_a

638

In [16]:
puzzle.answer_a = answer_a

## Part Two


In [20]:
print(puzzle.examples[0].input_data)

0123
1234
8765
9876


In [42]:
def solution_b(data: str) -> tuple[str, str, int]:
    # The paper describes a second way to measure a trailhead called its rating. 
    # A trailhead's rating is the number of distinct hiking trails which begin at that trailhead.
    hiking_map = np.array([list(map(int, list(line))) for line in data.splitlines()])
    n, c = hiking_map.shape
    print(hiking_map)
    score = 0
    
    trailheads = np.argwhere(hiking_map == 0)

    def find_trails(start_pos):
        complete_paths = []
        ongoing_paths = deque([[start_pos]])
    
        while ongoing_paths :
            print(ongoing_paths)
            current_path = ongoing_paths.pop()
            current_pos = current_path[-1]
            height = hiking_map[tuple(current_pos)]
    
            for x,y in [(-1,0), (1,0), (0,-1), (0,1)]:
                new_pos = current_pos + np.array([x,y])
    
                if 0 <= new_pos[0] < n and 0 <= new_pos[1] < c:
                    new_path = current_path + [new_pos]
                    if hiking_map[tuple(new_pos)] == 9 and height==8:
                        complete_paths.append(new_path)
                    elif hiking_map[tuple(new_pos)] == height + 1:
                        ongoing_paths.append(new_path)

        return complete_paths

    paths_per_trailhead = [(trailhead, find_trails(trailhead)) for trailhead in trailheads]

    # print out reachable peaks
    hiking_map = hiking_map.astype(str)
    for trailhead, paths in paths_per_trailhead:
        for path in paths:
            for step in path:
                hiking_map[tuple(step)] = "X"
    print(hiking_map)

    return sum(len(x[1]) for x in paths_per_trailhead)

In [43]:
solution_b(puzzle.examples[0].input_data)

[[0 1 2 3]
 [1 2 3 4]
 [8 7 6 5]
 [9 8 7 6]]
deque([[array([0, 0])]])
deque([[array([0, 0]), array([1, 0])], [array([0, 0]), array([0, 1])]])
deque([[array([0, 0]), array([1, 0])], [array([0, 0]), array([0, 1]), array([1, 1])], [array([0, 0]), array([0, 1]), array([0, 2])]])
deque([[array([0, 0]), array([1, 0])], [array([0, 0]), array([0, 1]), array([1, 1])], [array([0, 0]), array([0, 1]), array([0, 2]), array([1, 2])], [array([0, 0]), array([0, 1]), array([0, 2]), array([0, 3])]])
deque([[array([0, 0]), array([1, 0])], [array([0, 0]), array([0, 1]), array([1, 1])], [array([0, 0]), array([0, 1]), array([0, 2]), array([1, 2])], [array([0, 0]), array([0, 1]), array([0, 2]), array([0, 3]), array([1, 3])]])
deque([[array([0, 0]), array([1, 0])], [array([0, 0]), array([0, 1]), array([1, 1])], [array([0, 0]), array([0, 1]), array([0, 2]), array([1, 2])], [array([0, 0]), array([0, 1]), array([0, 2]), array([0, 3]), array([1, 3]), array([2, 3])]])
deque([[array([0, 0]), array([1, 0])], [array(

16

In [165]:
solution_b(puzzle.examples[0].input_data)[1].strip(".") == "00992111777.44.333....5555.6666.....8888"

00...111...2...333.44.5555.6666.777.888899
0099.111...2...333.44.5555.6666.777.8888..
0099.111...2...333.44.5555.6666.777.8888..
0099.1117772...333.44.5555.6666.....8888..
0099.1117772...333.44.5555.6666.....8888..
0099.1117772...333.44.5555.6666.....8888..
0099.111777244.333....5555.6666.....8888..
0099.111777244.333....5555.6666.....8888..
0099.111777244.333....5555.6666.....8888..
00992111777.44.333....5555.6666.....8888..
00992111777.44.333....5555.6666.....8888..
00992111777.44.333....5555.6666.....8888..
00992111777.44.333....5555.6666.....8888..


True

In [168]:
assert solution_b(puzzle.examples[0].input_data)[2] == 2858

In [ ]:
answer_b = solution_b(puzzle.input_data)
answer_b

In [170]:
puzzle.answer_b = answer_b[2]

That's the right answer!  You are one gold star closer to finding the Chief Historian.You have completed Day 9! You can [Shareon
  Bluesky
Twitter
Mastodon] this victory or [Return to Your Advent Calendar].
